In [1]:
import os
from pathlib import Path

import warnings
warnings.filterwarnings("ignore")

In [2]:
os.chdir("../src/")
%pwd

'/home/jovyan/nfs-home/scalable_ml_pipelines/src'

In [3]:
import socket

LOCAL_IP = socket.gethostbyname(socket.gethostname())

In [4]:
name_space = "eabraham-373705"

# Master node
kubernetes_master_url = "k8s://https://10.32.7.103:6443"

# Resource settings
driver_cores = "8"
executor_cores = "8"
driver_memory = "30g"
executor_memory = "30g"
executor_memory_overhead = "2g"

# These are the limits
cpu_limit = "3"  # 12 cores
memory_limit = "32g"  # Upto 32 GB
executor_limit = "8"



In [5]:
from pyspark.sql import SparkSession

APP_NAME = 'scalables_executor'


spark = SparkSession\
    .builder\
    .appName(APP_NAME)\
    .master(kubernetes_master_url)\
    .config("spark.driver.host", LOCAL_IP)\
    .config("spark.driver.bindAddress", "0.0.0.0")\
    .config("spark.executor.instances", "2")\
    .config("spark.executor.cores", executor_cores)\
    .config("spark.executor.memory", executor_memory)\
    .config("spark.memory.fraction", "0.8")\
    .config("spark.memory.storageFraction", "0.2")\
    .config("spark.sql.autoBroadcastJoinThreshold", "250MB")\
    .config("spark.kubernetes.executor.limit.cores", executor_limit)\
    .config("spark.kubernetes.namespace", name_space)\
    .config("spark.kubernetes.authenticate.driver.serviceAccountName", "spark")\
    .config("spark.kubernetes.driver.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.label.appname", APP_NAME)\
    .config("spark.kubernetes.executor.deleteOnTermination", "false") \
    .config("spark.kubernetes.container.image.pullPolicy", "Always") \
    .config("spark.kubernetes.container.image", "node03.st:5000/pyspark-hdfs-jupyter:eabraham-373705-v4-executor")\
    .config("spark.local.dir", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.driver.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.path", "/tmp/spark")\
    .config("spark.kubernetes.executor.volumes.emptyDir.spark-local-dir-tmp-spark.mount.readOnly", "false")\
    .getOrCreate()


23/12/21 16:49:08 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/12/21 16:49:09 WARN spark.SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [6]:
spark

In [7]:
from us_used_cars_ml_pipeline.components.data_preparation import DataPreparation
# from us_used_cars_ml_pipeline.utils.common import get_spark_session
from us_used_cars_ml_pipeline.config.configuration import ConfigurationManager

config_manager = ConfigurationManager()
data_preparation_config = config_manager.get_data_preparation_config()

data_preparation = DataPreparation(data_preparation_config)

[2023-12-21 16:49:16,735: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/config/config.yaml loaded successfully]
[2023-12-21 16:49:16,739: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/params.yaml loaded successfully]
[2023-12-21 16:49:16,741: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/schema.yaml loaded successfully]


In [8]:
data_preparation_config

DataPreparationConfig(path_to_cleaned_data='hdfs:///home/eabraham-373705/data/cleaned/', path_to_prepared_data='hdfs:///home/eabraham-373705/data/prepared/', scaler_name='MinMaxScaler', path_to_scalers='hdfs:///home/eabraham-373705/models/scalers/', path_to_features_list='us_used_cars_ml_pipeline/constants/features_list_for_data_preparation.yaml', path_to_imputers='hdfs:///home/eabraham-373705/models/imputers/', path_to_scores_file='us_used_cars_ml_pipeline/tests/imputers_training_scores.yaml', models=ConfigBox({'LogisticRegression': {'maxIter': 100, 'regParam': 1, 'elasticNetParam': 0.5}, 'LinearRegression': {'maxIter': 100, 'regParam': 1, 'elasticNetParam': 0.5}, 'RandomForestRegressor': {'maxDepth': 5, 'numTrees': 50, 'seed': 42}}))

In [9]:
%%time
data_preparation.run_stage(spark, is_new_data=False)

[2023-12-21 16:49:28,433: 468: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  Cleaned data has been read]
[2023-12-21 16:49:28,434: 471: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  PART I. STARTING]
[2023-12-21 16:49:28,450: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_list_for_data_preparation.yaml loaded successfully]


23/12/21 16:49:29 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[2023-12-21 16:49:38,618: 323: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  1. features_for_imputers group has been scaled]
[2023-12-21 16:49:38,658: 327: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  2. Data has been merged]


Py4JJavaError: An error occurred while calling o208.count.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange SinglePartition, ENSURE_REQUIREMENTS, [id=#103]
+- *(1) HashAggregate(keys=[], functions=[partial_count(1)], output=[count#1139L])
   +- InMemoryTableScan
         +- InMemoryRelation [ID#16L, features_for_imputers#516, back_legroom#75, city_fuel_economy#76, labelencoding_fleet#77, labelencoding_frame_damaged#78, labelencoding_has_accidents#79, highway_fuel_economy#80, labelencoding_salvage#81, torque_value#82, torque_rpm#83, power_value#84, power_rpm#85, owner_count#86, Backup_Camera#87, Bluetooth#88, Alloy_Wheels#89, Heated_Seats#90, Navigation_System#91, Sunroof_Moonroof#92, Remote_Start#93, CarPlay#94, Android_Auto#95, Leather_Seats#96, ... 4 more fields], StorageLevel(disk, memory, deserialized, 1 replicas)
               +- *(2) Project [ID#16L, features_for_imputers#516, back_legroom#75, city_fuel_economy#76, labelencoding_fleet#77,

In [9]:
%%time
df1 = data_preparation.read_data_from_hdfs(spark, 'features_for_imputers.parquet')
df2 = data_preparation.read_data_from_hdfs(spark, 'features_with_nans.parquet')
df3 = data_preparation.read_data_from_hdfs(spark, 'other_features.parquet')

data_preparation.prepare_part2(df1, df2, df3, is_new_data=False, as_separate=True)

[2023-12-15 15:36:47,928: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_list_for_data_preparation.yaml loaded successfully]


[2023-12-15 15:36:50,786: 380: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  6. features_for_imputers group has been scaled]


  0%|          | 0/18 [00:00<?, ?it/s]

[2023-12-15 15:36:59,959: 390: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  7.1 Missing data in categorical features has been filled in]


  0%|          | 0/8 [00:00<?, ?it/s]

...

23/12/15 15:37:09 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[2023-12-15 15:37:19,602: 407: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.1.2. numerical_features_with_nans group has been scaled]
[2023-12-15 15:37:19,658: 398: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.2.1. categorical_features_with_nans features has been grouped]


[2023-12-15 15:37:28,999: 407: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.2.2. categorical_features_with_nans group has been scaled]
[2023-12-15 15:37:29,166: 412: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  9. other_features features has been grouped]
[2023-12-15 15:37:29,188: 416: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  10. Data has been merged]
[2023-12-15 15:37:29,312: 425: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  11. Data has been regrouped]


23/12/15 15:37:36 ERROR datasources.FileFormatWriter: Aborting job 0fe9e8db-f1c5-4486-bf81-720257515716.
java.util.concurrent.ExecutionException: org.apache.spark.util.SparkFatalException
	at java.base/java.util.concurrent.FutureTask.report(FutureTask.java:122)
	at java.base/java.util.concurrent.FutureTask.get(FutureTask.java:205)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:194)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark

Py4JJavaError: An error occurred while calling o3302.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache

In [9]:
%%time
df1 = data_preparation.read_data_from_hdfs(spark, 'features_for_imputers.parquet')
df2 = data_preparation.read_data_from_hdfs(spark, 'features_with_nans.parquet')
df3 = data_preparation.read_data_from_hdfs(spark, 'other_features.parquet')

df1 = data_preparation.prepare_part2(df1, df2, df3, is_new_data=False, as_separate=True)

[2023-12-15 16:37:40,832: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_list_for_data_preparation.yaml loaded successfully]


[2023-12-15 16:37:43,790: 380: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  6. features_for_imputers group has been scaled]


  0%|          | 0/18 [00:00<?, ?it/s]

[2023-12-15 16:37:52,740: 390: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  7.1 Missing data in categorical features has been filled in]


  0%|          | 0/8 [00:00<?, ?it/s]

[2023-12-15 16:38:00,543: 390: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  7.2 Missing data in numerical features has been filled in]
[2023-12-15 16:38:00,623: 398: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.1.1. numerical_features_with_nans features has been grouped]


23/12/15 16:38:00 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


[2023-12-15 16:38:11,128: 407: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.1.2. numerical_features_with_nans group has been scaled]
[2023-12-15 16:38:11,186: 398: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.2.1. categorical_features_with_nans features has been grouped]


[2023-12-15 16:38:20,206: 407: us_used_cars_ml_pipeline_logger: INFO: data_preparation:  8.2.2. categorical_features_with_nans group has been scaled]
CPU times: user 501 ms, sys: 244 ms, total: 746 ms
Wall time: 45 s


In [10]:
df1.printSchema()

root
 |-- ID: long (nullable = true)
 |-- features_for_imputers: vector (nullable = true)
 |-- numerical_features_with_nans: vector (nullable = true)
 |-- categorical_features_with_nans: vector (nullable = true)



In [11]:
df1.show()

+----+---------------------+----------------------------+------------------------------+
|  ID|features_for_imputers|numerical_features_with_nans|categorical_features_with_nans|
+----+---------------------+----------------------------+------------------------------+
|  26| [0.0,0.1469852736...|        [0.58528428840362...|          (18,[0,1,2],[1.0,...|
|  29| [0.0,0.0544595721...|        [0.62876252431151...|          (18,[3,4,5,12],[1...|
| 474| [0.0,0.0022228396...|        [0.61204011607141...|          (18,[0,1,2,3,4,5,...|
| 964| [0.125,0.00472353...|        [0.62876252431151...|          (18,[4,6,14,15],[...|
|1677| [0.0,0.0072242289...|        [0.65217392136403...|          (18,[0,3,4,5,7,8,...|
|1697| [0.0,0.0739094192...|        [0.64381271724398...|          (18,[0,1,2,4,5,6,...|
|1806| [0.625,0.00889135...|        [0.55351168723105...|          (18,[0,1,2,3,4,6,...|
|1950| [1.0,2.7785495971...|        [0.52341136515705...|          (18,[0,1,3,4,6,9]...|
|2040| [0.125,0.02445

In [15]:
from pyspark.sql import functions as F
from pyspark.ml.functions import vector_to_array
from pyspark.sql.types import FloatType, ArrayType

df1 = df1.select('ID', F.concat(vector_to_array(F.col('features_for_imputers')).cast(ArrayType(FloatType())),
                                vector_to_array(F.col('numerical_features_with_nans')).cast(ArrayType(FloatType())), 
                                vector_to_array(F.col('categorical_features_with_nans')).cast(ArrayType(FloatType()))).alias('features'))

In [16]:
df1.printSchema()

root
 |-- ID: long (nullable = true)
 |-- features: array (nullable = false)
 |    |-- element: float (containsNull = true)



In [17]:
df1.show()

+----+--------------------+
|  ID|            features|
+----+--------------------+
|  26|[0.0, 0.14698528,...|
|  29|[0.0, 0.05445957,...|
| 474|[0.0, 0.002222839...|
| 964|[0.125, 0.0047235...|
|1677|[0.0, 0.007224229...|
|1697|[0.0, 0.07390942,...|
|1806|[0.625, 0.0088913...|
|1950|[1.0, 2.7785497E-...|
|2040|[0.125, 0.0244512...|
|2214|[0.0, 5.5570994E-...|
|2250|[0.0, 0.001111419...|
|2453|[0.125, 0.0144484...|
|2509|[0.125, 0.0080577...|
|2529|[0.125, 0.0150041...|
|2927|[0.125, 0.0147263...|
|3091|[0.125, 0.0544595...|
|3506|[0.0, 0.002778549...|
|3764|[0.125, 0.0647402...|
|4590|[0.125, 0.0058349...|
|4823|[0.0, 0.004167824...|
+----+--------------------+
only showing top 20 rows



In [18]:
data_preparation.save_data(df1, 'prepared_data_1.parquet', is_new_data=False) 

In [9]:
df1 = spark.read.parquet('/home/eabraham-373705/data/prepared/prepared_data_1.parquet', header=True, inferSchema=True)
df3 = data_preparation.read_data_from_hdfs(spark, 'other_features.parquet')

In [10]:
from us_used_cars_ml_pipeline.utils.common import read_yaml
from pyspark.sql import functions as F

features = read_yaml(Path(data_preparation.config.path_to_features_list))

other_features = [k for k in features['other_features'].to_list() if k != 'price']
df3 = df3.select('ID', 'price', F.array(*other_features).alias('other_features'))

df1 = df1.join(df3, on='ID', how='left')

[2023-12-15 17:09:26,141: 44: us_used_cars_ml_pipeline_logger: INFO: common:  yaml file: us_used_cars_ml_pipeline/constants/features_list_for_data_preparation.yaml loaded successfully]


In [15]:
df1 = df1.select('ID', 'price', F.concat(F.col('features'), F.col('other_features')).alias('features'))

In [11]:
df1.show()

23/12/15 17:09:30 WARN util.package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Py4JJavaError: An error occurred while calling o271.showString.
: java.util.concurrent.ExecutionException: org.apache.spark.util.SparkFatalException
	at java.base/java.util.concurrent.FutureTask.report(FutureTask.java:122)
	at java.base/java.util.concurrent.FutureTask.get(FutureTask.java:205)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:194)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark.sql.execution.joins.BroadcastHashJoinE

In [18]:
data_preparation.save_data(df1, 'prepared_data.parquet', is_new_data=False)  

23/12/15 17:06:19 ERROR datasources.FileFormatWriter: Aborting job 9ab8c5f4-7f11-4ee6-aaec-00dfee936a4f.
java.util.concurrent.ExecutionException: org.apache.spark.util.SparkFatalException
	at java.base/java.util.concurrent.FutureTask.report(FutureTask.java:122)
	at java.base/java.util.concurrent.FutureTask.get(FutureTask.java:205)
	at org.apache.spark.sql.execution.exchange.BroadcastExchangeExec.doExecuteBroadcast(BroadcastExchangeExec.scala:194)
	at org.apache.spark.sql.execution.InputAdapter.doExecuteBroadcast(WholeStageCodegenExec.scala:515)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeBroadcast$1(SparkPlan.scala:193)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.spark.sql.execution.SparkPlan.executeBroadcast(SparkPlan.scala:189)
	at org.apache.spark

Py4JJavaError: An error occurred while calling o299.save.
: org.apache.spark.SparkException: Job aborted.
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:231)
	at org.apache.spark.sql.execution.datasources.InsertIntoHadoopFsRelationCommand.run(InsertIntoHadoopFsRelationCommand.scala:188)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult$lzycompute(commands.scala:108)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.sideEffectResult(commands.scala:106)
	at org.apache.spark.sql.execution.command.DataWritingCommandExec.doExecute(commands.scala:131)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$execute$1(SparkPlan.scala:180)
	at org.apache.spark.sql.execution.SparkPlan.$anonfun$executeQuery$1(SparkPlan.scala:218)
	at org.apache.spark.rdd.RDDOperationScope$.withScope(RDDOperationScope.scala:151)
	at org.apache.spark.sql.execution.SparkPlan.executeQuery(SparkPlan.scala:215)
	at org.apache.

In [12]:
df3.show()

+------------+--------+--------------------+
|          ID|   price|      other_features|
+------------+--------+--------------------+
|103079286797| 20895.0|[1.0, 1.0, 1.0, 1...|
|214748369828|417425.0|[0.0, 1.0, 1.0, 0...|
| 42949786706|355880.0|[0.0, 1.0, 1.0, 1...|
|137439039784|  8699.0|[1.0, 1.0, 1.0, 1...|
|111669177308|220995.0|[1.0, 1.0, 1.0, 1...|
|214748393181| 14995.0|[1.0, 1.0, 1.0, 1...|
| 42949710780|  8597.0|[1.0, 1.0, 1.0, 1...|
|111669151626|  5980.0|[1.0, 0.0, 1.0, 1...|
|  8589958953|  7984.0|[1.0, 0.0, 1.0, 1...|
|137438975993|  7497.0|[1.0, 1.0, 1.0, 1...|
|180388626869| 46353.0|[1.0, 1.0, 1.0, 0...|
| 60129606862|  8749.0|[1.0, 0.0, 1.0, 1...|
|180388626777| 46262.0|[1.0, 1.0, 1.0, 1...|
| 42949787262|259841.0|[1.0, 1.0, 1.0, 1...|
|  8589999204|545200.0|[1.0, 1.0, 1.0, 0...|
| 60129642352|  9998.0|[1.0, 0.0, 1.0, 1...|
|163208801675| 53292.0|[0.0, 0.0, 1.0, 0...|
| 34359815424| 16500.0|[1.0, 0.0, 1.0, 1...|
| 51539716167| 54715.0|[0.0, 0.0, 1.0, 0...|
|103079309

In [13]:
data_preparation.save_data(df3, 'prepared_data_2.parquet', is_new_data=False) 

In [9]:
df1 = spark.read.parquet('/home/eabraham-373705/data/prepared/prepared_data_1.parquet', header=True, inferSchema=True)
df3 = spark.read.parquet('/home/eabraham-373705/data/prepared/prepared_data_2.parquet', header=True, inferSchema=True)

In [10]:
df1 = df1.join(df3, on='ID', how='left')

In [12]:
df1.show()

+----+--------------------+-------+--------------------+
|  ID|            features|  price|      other_features|
+----+--------------------+-------+--------------------+
|  26|[0.0, 0.14698528,...|24245.0|[1.0, 0.0, 1.0, 0...|
|  29|[0.0, 0.05445957,...|68950.0|[1.0, 0.0, 1.0, 0...|
| 474|[0.0, 0.002222839...|28000.0|[1.0, 0.0, 1.0, 1...|
| 964|[0.125, 0.0047235...| 5750.0|[1.0, 1.0, 1.0, 1...|
|1677|[0.0, 0.007224229...|50060.0|[1.0, 1.0, 1.0, 0...|
|1697|[0.0, 0.07390942,...|37195.0|[1.0, 1.0, 1.0, 1...|
|1806|[0.625, 0.0088913...|36993.0|[1.0, 1.0, 1.0, 1...|
|1950|[1.0, 2.7785497E-...|14495.0|[1.0, 1.0, 1.0, 1...|
|2040|[0.125, 0.0244512...|27845.0|[1.0, 1.0, 1.0, 0...|
|2214|[0.0, 5.5570994E-...|24995.0|[1.0, 1.0, 1.0, 1...|
|2250|[0.0, 0.001111419...|33500.0|[1.0, 1.0, 1.0, 0...|
|2453|[0.125, 0.0144484...|35791.0|[1.0, 1.0, 1.0, 1...|
|2509|[0.125, 0.0080577...|14995.0|[1.0, 1.0, 1.0, 1...|
|2529|[0.125, 0.0150041...|16705.0|[1.0, 1.0, 1.0, 1...|
|2927|[0.125, 0.0147263...|1760

In [14]:
from pyspark.sql import functions as F

df1 = df1.select('ID', 'price', F.concat(F.col('features'), F.col('other_features')).alias('features'))

In [15]:
df1

DataFrame[ID: bigint, price: float, features: array<float>]

In [16]:
data_preparation.save_data(df1, 'prepared_data.parquet', is_new_data=False) 